In [2]:
#定义网络
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):  

    def __init__(self):
        super(Net, self).__init__()
        #1 input image channel, 6 output channels, 3x3 square convolution 设置第一个卷积核 C1 ，1个通道的输入，6个通道的输出，
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)),(2, 2))
        #If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) #view函数将张量x变形成一维的向量形式，总特征数并不改变，为接下来的全连接作准备
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension，这里为什么要使用[1:],是因为pytorch只接受批输入，也就是说一次性输入好几张图片，那么输入数据张量的维度自然上升到了4维。【1:】让我们把注意力放在后3维上面
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [6]:
#返回模型的学习参数
params = list(net.parameters())
print(len(params))
print(params[0].size())  #conv1 的权重尺寸

10
torch.Size([6, 1, 3, 3])


In [7]:
#尝试输入一个随机大小为 32x32 的数据
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0112,  0.1028,  0.0030,  0.1391,  0.0997, -0.0673,  0.0512,  0.0096,
         -0.0556,  0.1668]], grad_fn=<AddmmBackward>)


In [9]:
#用随机梯度将所有参数和反向传播器的梯度缓冲区归零
net.zero_grad()
out.backward(torch.randn(1, 10))

In [12]:
#损失函数，举一个简单的损失是：nn.MSELoss 计算输入和目标之间的均方差
output = net(input)
target = torch.randn(10)  #随机取一个 10维的target
print(target)
target = target.view(1, -1) #使target变成（1,10）维的张量，和输出的维度一样
print(target)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor([ 0.5855, -0.5268, -0.0837,  1.2937,  0.0978, -0.8014,  0.0687, -1.5697,
         0.1794, -1.3178])
tensor([[ 0.5855, -0.5268, -0.0837,  1.2937,  0.0978, -0.8014,  0.0687, -1.5697,
          0.1794, -1.3178]])
tensor(0.7386, grad_fn=<MseLossBackward>)


In [13]:
#反向传播，使用loss.backward(),不过需要清除现有的渐变，否则渐变将累加到现有的渐变中
net.zero_grad()

print('反向传播之前的 conv1.bias.grad')
print(net.conv1.bias.grad)

loss.backward()

print('反向传播之后的 conv1.bias.grad')
print(net.conv1.bias.grad)

反向传播之前的 conv1.bias.grad
tensor([0., 0., 0., 0., 0., 0.])
反向传播之后的 conv1.bias.grad
tensor([ 0.0057,  0.0073, -0.0172, -0.0094,  0.0046,  0.0030])


In [15]:
#权重的更新  SGD: weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
    

In [16]:
#使用不同的权重更新规则：SGD, Nesterov-SGD, Adam, RMSProp 。可以使用小程序包：torch.optim 来实现
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)  #创建optimizer

#在你的训练层
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()   #更新权重